In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib
import os

# Path where models are saved
model_path = '/content/drive/MyDrive/Stable_XAI_Saved_models/'

# Scenario 1: Galaxy Morphology Classification
model_s1 = joblib.load(os.path.join(model_path, 'SDSS_S1_USC_LRC.pkl'))

# Scenario 2: Galaxy Redshift Estimation
model_s2 = joblib.load(os.path.join(model_path, 'SDSS_S2_Reg_XGB.pkl'))
median_s2 = joblib.load(os.path.join(model_path, 'SDSS_feature_medians.pkl'))
columns_s2 = joblib.load(os.path.join(model_path, 'SDSS_feature_columns.pkl'))

# Scenario 3: AGN Identification
model_s3 = joblib.load(os.path.join(model_path, 'SDSS_S3_USC_XGB.pkl'))

print("All models and preprocessing artifacts loaded successfully.")


✅ All models and preprocessing artifacts loaded successfully.


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Example manual input (replace values with real ones or approximate SDSS ranges)
sample_s1 = pd.DataFrame({
    "u_g": [1.2],
    "g_r": [0.7],
    "r_i": [0.4],
    "i_z": [0.3],
    "petroR50_g": [5.1],
    "petroR50_r": [5.0],
    "petroR50_i": [4.8],
    "expAB_g": [0.7],
    "expAB_r": [0.8],
    "expAB_i": [0.75]
})

# Scale the input using same approach as training
scaler_s1 = StandardScaler()
sample_s1_scaled = scaler_s1.fit_transform(sample_s1)  # optionally use the scaler used in training
sample_s1_scaled_df = pd.DataFrame(sample_s1_scaled, columns=sample_s1.columns)
# Predict
pred_s1 = model_s1.predict(sample_s1_scaled)
print("Predicted morphology cluster label:", pred_s1)


Predicted morphology cluster label: [1]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
# sample input with exact columns as training
sample_s2 = pd.DataFrame(np.zeros((1, len(columns_s2))), columns=columns_s2)

# Example: fill some values manually (replace with real approximate values)
sample_s2['u'] = 17.5
sample_s2['g'] = 16.8
sample_s2['r'] = 16.3
sample_s2['i'] = 16.0
sample_s2['z'] = 15.9

# Fill missing values with medians
sample_s2 = sample_s2.fillna(median_s2)

# Predict
pred_s2 = model_s2.predict(sample_s2)
print("Predicted redshift:", pred_s2[0])


Predicted redshift: 0.07616078


In [ ]:
# Example manual input
sample_s3 = pd.DataFrame([{
    'u': 17.5, 'g': 16.8, 'r': 16.3, 'i': 16.0, 'z': 15.9,
    'psfMag_u': 17.4, 'psfMag_g': 16.7, 'psfMag_r': 16.2, 'psfMag_i': 15.9, 'psfMag_z': 15.8,
    'modelFlux_u': 1.0, 'modelFlux_g': 1.2, 'modelFlux_r': 1.1, 'modelFlux_i': 1.0, 'modelFlux_z': 0.9,
    'u_g': 0.7, 'g_r': 0.5, 'r_i': 0.3, 'i_z': 0.1,
    'compact_u': 0.1, 'compact_g': 0.1, 'compact_r': 0.1, 'compact_i': 0.1, 'compact_z': 0.1,
    'flux_u_g': 0.83, 'flux_g_r': 1.09, 'flux_r_i': 1.1, 'flux_i_z': 1.05
}])

# Scale using StandardScaler
scaler_s3 = StandardScaler()
sample_s3_scaled = scaler_s3.fit_transform(sample_s3)  # optionally use the training scaler

# Predict
pred_s3 = model_s3.predict(sample_s3_scaled)
pred_label_s3 = 'AGN' if pred_s3[0] == 1 else 'Non-AGN'
print("Predicted AGN label:", pred_label_s3)


Predicted AGN label: AGN


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Step 1: Raw input
sample_s3 = pd.DataFrame([{
    'u': 17.5, 'g': 16.8, 'r': 16.3, 'i': 16.0, 'z': 15.9,
    'psfMag_u': 17.4, 'psfMag_g': 16.7, 'psfMag_r': 16.2, 'psfMag_i': 15.9, 'psfMag_z': 15.8,
    'modelFlux_u': 1.0, 'modelFlux_g': 1.2, 'modelFlux_r': 1.1, 'modelFlux_i': 1.0, 'modelFlux_z': 0.9
}])

# Step 2: Compute derived features exactly like in training
sample_s3['u_g'] = sample_s3['u'] - sample_s3['g']
sample_s3['g_r'] = sample_s3['g'] - sample_s3['r']
sample_s3['r_i'] = sample_s3['r'] - sample_s3['i']
sample_s3['i_z'] = sample_s3['i'] - sample_s3['z']

for band in ['u','g','r','i','z']:
    sample_s3[f'compact_{band}'] = sample_s3[f'psfMag_{band}'] - sample_s3[f'modelFlux_{band}']

sample_s3['flux_u_g'] = sample_s3['modelFlux_u'] / (sample_s3['modelFlux_g'] + 1e-5)
sample_s3['flux_g_r'] = sample_s3['modelFlux_g'] / (sample_s3['modelFlux_r'] + 1e-5)
sample_s3['flux_r_i'] = sample_s3['modelFlux_r'] / (sample_s3['modelFlux_i'] + 1e-5)
sample_s3['flux_i_z'] = sample_s3['modelFlux_i'] / (sample_s3['modelFlux_z'] + 1e-5)

# Step 3: Scale
scaler_s3 = StandardScaler()
sample_s3_scaled = scaler_s3.fit_transform(sample_s3)

# Step 4: Predict
pred_s3 = model_s3.predict(sample_s3_scaled)
pred_label_s3 = 'AGN' if pred_s3[0] == 1 else 'Non-AGN'
print("Predicted AGN label:", pred_label_s3)


Predicted AGN label: AGN
